## About
This is a notebook for calculating the following statistics in the iceplant map:
- producer's accuracies and user's accuracies by class,
- overall accuracy, and
- area estimates. 

Each statistic is accompanied by a 95% confidence interval. 
To do this, we follow the notation and calculations in Olofsson et al., 2014.

The notebook uses the following files:
1. A CSV file of the sampled points used for the accuracy assessment (each point is a row) with two columns: `map_class` and `ref_class`. The `map_class` column is the classification of a given point in the map, while `ref_class` is the \"ground truth\" classification of the point.

2. A CSV file with the number of pixels per class in the map.

## References

Pontus Olofsson, Giles M. Foody, Martin Herold, Stephen V. Stehman, Curtis E. Woodcock, Michael A. Wulder, Good practices for estimating area and assessing accuracy of land change, Remote Sensing of Environment,Volume 148, 2014, Pages 42-57, ISSN 0034-4257, https://doi.org/10.1016/j.rse.2014.02.015.  (https://www.sciencedirect.com/science/article/pii/S0034425714000704)

Perrot, M., & Duchesnay, É. (2011). Scikit-learn: Machine Learning in Python. Journal of Machine Learning Research, 12, 2825--2830.

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

# Assuming repository's parent directory is the Documents directory
home = os.path.expanduser("~")
os.chdir(os.path.join(home,'Documents','iceplant-detection-santa-barbara'))

### Load Data

In [21]:
# number of classes in the map
n_classes = 3

validation_data_dir = os.path.join(os.getcwd(),
                              'data',
                              'validation_data')

# load csv with validation points
df = pd.read_csv(os.path.join(validation_data_dir,
                              'final_model_map_and_reference_classes.csv'))

# load counts of pixels per class in map
pixel_count_path = os.path.join(validation_data_dir,
                              'map_pixel_counts',
                              'final_model_combined_pixel_counts_total.csv')

pix_counts = pd.read_csv(pixel_count_path)
pix_counts = pix_counts.to_numpy()[0]

print("Pixel counts per class:")
for n_pix, i  in zip(pix_counts, range(n_classes)):
    print(f'class {i+1}: {pix_counts[i]:>12,}')

Pixel counts per class:
class 1:  120,173,466
class 2:    5,981,423
class 3:  188,071,487


In [26]:
# counts by reference class
print('Points in each reference class')
counts = np.unique(df.ref_class, return_counts=True)
for i, n_pts in zip(counts[0],counts[1]):
    print(f'class {i+1}: {n_pts}')
print('\n')

# counts by map class: these should match the counts given by the stratified sample design
print('Points in each map class')
counts = np.unique(df.map_class, return_counts=True)
for i, n_pts in zip(counts[0],counts[1]):
    print(f'class {i+1}: {n_pts}')

Points in each reference class
class 1: 321
class 2: 264
class 3: 404


Points in each map class
class 1: 289
class 2: 296
class 3: 404


### Confusion Matrix

Here we create a confusion matrix $n$ such that 

$n_{i,j}$ = number of points predicted as $i$, known to be $j$, 

which is equivalent to

$n_{i,j}$ = number of points that have map class as $i$ and reference class $j$.

To do this, we use the [`confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) function from the scikit-learn Python package (Perrot & Duchesnay, 2011). 
The matrix $C$ obtained with `confusion_matrix` is such that

$C_{i,j}$ = points known to belong to class $i$ and are 
predicted as class  $j$.

However, the notation in the Olofsson et al. paper is

$n_{i,j}$ = number of points predicted as class $i$, known to be class $j$,

so we need to take the transpose of $C$ to match the notation.

In [4]:
# calculate confusion matrix pro imported data
n = confusion_matrix(df.ref_class, df.map_class, labels=range(n_classes)).T
n

array([[247,   2,  40],
       [ 26, 262,   8],
       [ 48,   0, 356]])

### User's Accuracy

In [5]:
# points in sample that had class i in map (predicted as i, any true class j)
n_idot = [sum(n[i,:]) for i in range(n_classes)]

# -------------------------------------
# estimated user's accuracy
U_hat = [n[i,i] / n_idot[i] for i in range(n_classes)]

# variance of estimated user's accuracy
var_U_hat = [U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(n_classes)]

# -------------------------------------
print("User's accuracies with 95% confidence interval:")
for U_hati, var_i, i  in zip(U_hat, var_U_hat, range(n_classes)):
    print(f'class {i+1}: {U_hati*100:.2f} ± {1.96 * np.sqrt(var_i)*100:.2f}')

User's accuracies with 95% confidence interval:
class 1: 85.47 ± 4.07
class 2: 88.51 ± 3.64
class 3: 88.12 ± 3.16


### Overall Accuracy

In [6]:
# total number of pixels in the map
total_pix = sum(pix_counts)

# fractions of area in map classified in each class
W = [pix_counts[i]/ total_pix for i in range(n_classes)]      

# -------------------------------------
# estimated overall accuracy
O_hat = sum([W[i]*n[i,i]/n_idot[i] for i in range(n_classes)])

# -------------------------------------
# variance of estimated overall accuracy
var_O_hat = sum([ W[i]**2 * U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(n_classes)])

# -------------------------------------
print('Overall accuracy with 95% confidence interval:')
print(f'{O_hat*100:.2f} ± {1.96*np.sqrt(var_O_hat)*100:.2f}')

Overall accuracy with 95% confidence interval:
87.11 ± 2.45


### Producer's Accuracy

In [7]:
# estimated fraction of area in class j (Olofsson et al., eq. 9) 
p_hat_dotj = []
for j in range(n_classes):
    p_hat_ij = [ W[i]*n[i,j]/n_idot[i] for i in range(n_classes) ]
    p_hat_dotj.append(sum(p_hat_ij))  
    
# -------------------------------------
# estimated producer's accuracy 
P_hat = [ (W[j]*n[j,j]/n_idot[j]) / p_hat_dotj[j] for j in range(n_classes)]

# -------------------------------------
# variance of estimated producer's accuracy 
# notice N_jdot is pix_counts[j]
N_hat_dotj = []
for j in range(n_classes):
    summands = [ pix_counts[i] * n[i,j]/n_idot[i] for i in range(n_classes)]
    N_hat_dotj.append(sum(summands))

summand1 = []
for j in range(n_classes):
    summand1.append((pix_counts[j]**2) * ((1-P_hat[j])**2) * U_hat[j] * (1-U_hat[j]) / (n_idot[j] - 1))

summand2 = []
for j in range(n_classes):
    inner = []
    for i in range(n_classes):
        if i!=j:
            inner.append( (pix_counts[i]**2) * (n[i,j])/(n_idot[i]) * ( 1 - n[i,j]/n_idot[i])/(n_idot[i]-1)) 
    summand2.append((P_hat[j]**2) * sum(inner))

var_P_hat = [1/(N_hat_dotj[j]**2) *  (summand1[j] + summand2[j]) for j in range(n_classes)]

# -------------------------------------
print("Producer's accuracies with 95% confidence interval:")
for P_hati, var_i, i  in zip(P_hat, var_P_hat, range(n_classes)):
    print(f'class {i+1}: {P_hati*100:.2f} ± {1.96 * np.sqrt(var_i)*100:.2f}')

Producer's accuracies with 95% confidence interval:
class 1: 81.79 ± 3.94
class 2: 86.42 ± 16.24
class 3: 90.80 ± 2.40


### Area Estimates

In [8]:
# standard error of estimated area per class
S_p_hat_dotj = []
for j in range(n_classes):
    summands = [ (W[i]**2) * (n[i,j]/n_idot[i]) * (1 -  (n[i,j]/n_idot[i]))/ (n_idot[i]-1) 
                for i in range(n_classes)]
    S_p_hat_dotj.append(np.sqrt(sum(summands)))
    
# -------------------------------------
print("Percentage area per class with 95% confidence interval:")
for perc_i, err_i, i  in zip(p_hat_dotj, S_p_hat_dotj, range(n_classes)):
    print(f'class {i+1}: {perc_i*100:.2f} ± {1.96 * err_i*100:.2f}')

Percentage area per class with 95% confidence interval:
class 1: 39.96 ± 2.45
class 2: 1.95 ± 0.37
class 3: 58.09 ± 2.43


In [10]:
# area per pixel in m^2
pixel_area_m2 = 0.6**2

# total map area in km^2 (1 m^2 = 1/10^6 ha)
map_area_ha = total_pix * pixel_area_m2 / 10**6

approx_area_per_class = [map_area_ha * p_hat_dotj[i] for i in range(n_classes)]

# standard error area per class
SE_area_per_class = [map_area_ha * S_p_hat_dotj[i] for i in range(n_classes)]

# -------------------------------------
print("Estimated area per class (km^2) with 95% confidence interval:")
for area_i, err_i, i  in zip(approx_area_per_class, SE_area_per_class, range(n_classes)):
    print(f'class {i+1}: {area_i:,.2f} ± {1.96 * err_i:,.2f}')

Estimated area per class (km^2) with 95% confidence interval:
class 1: 45.21 ± 2.77
class 2: 2.21 ± 0.42
class 3: 65.71 ± 2.75
